<h1>Capstone - Week 3 (Skip to part 2)</h1>

<h2>Part 1:</h2>

In [21]:
#First, we import the necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

#Then we scrape the Wikipedia page for the required table data, and we save it in a .CSV file:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
tree = BeautifulSoup(source, 'lxml')

csv_file = open("table_data.csv","w",newline='')
csv_writer = csv.writer(csv_file)

table_tag = tree.select("table")[0]
tab_data = [[item.text.split('\n')[0] for item in row_data.select("th,td")]
                for row_data in table_tag.select("tr")]

for data in tab_data:
    csv_writer.writerow(data)

    
#We read our .CSV file into a dataframe:
df = pd.read_csv('table_data.csv')

#Remove all table rows without assigned boroughs:
df = df.drop(df.index[df.Borough == 'Not assigned'])

#Replace 'Not assigned' in Neighbourhood column with corresponding value from Borough column:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df['Borough']

#Group data by Postcode and join all neighbourhoods that exist in the same postal code area:
df = df.groupby('Postcode').agg(lambda x: ','.join(set(x))).reset_index()

#Finally, we print the final number of rows and columns:
df.shape

(103, 3)

In [22]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>Part 2:</h2>

<h4>First, we read the .CSV containg the longitude and latitude data into a new dataframe:</h4>

In [23]:
geodf = pd.read_csv('https://cocl.us/Geospatial_data')
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h4>Then, we add the columns from the new geo-dataframe to our existing dataframe:</h4>

In [24]:
df = df.join(geodf)
df.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
